# Predict Radiative heating corresponding to given parameters

## Import package

In [60]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

from metpy.calc import pressure_to_height_std
from metpy.units import units
from scipy.interpolate import interp1d

## Load data

In [61]:
# Load LRF
## Load LRF for T, q
with h5py.File("/work/b11209013/2025_Research/MSI/Rad_Stuff/LRF.h5", "r") as f:
    q_lw_lrf = np.array(f.get("q_lw"))
    q_sw_lrf = np.array(f.get("q_sw"))
    t_lw_lrf = np.array(f.get("t_lw"))
    t_sw_lrf = np.array(f.get("t_sw"))

## Load LRF for w
with h5py.File("/home/b11209013/2025_Research/Obs/Files/ERA5/LRF_SW_w.h5", "r") as f:
    w_sw_lrf = np.array(f.get("LRF"))
    eof = np.array(f.get("EOF"))
with h5py.File("/home/b11209013/2025_Research/Obs/Files/ERA5/LRF_LW_w.h5", "r") as f:
    w_lw_lrf = np.array(f.get("LRF"))

# load regressed moisture profile
q_reg = np.loadtxt("/work/b11209013/2025_Research/MSI/Rad_Stuff/mean_corr_moisture.txt")

# Load vertical modes
with h5py.File("/home/b11209013/2025_Research/Kuang2008/data/vertical_mode.h5", "r") as f:
    G1 = np.array(f.get("G1")).squeeze() 
    G2 = np.array(f.get("G2")).squeeze()
    z  = np.array(f.get("z"))

# Load background density profile
with h5py.File("/home/b11209013/2025_Research/Kuang2008/data/background.h5","r") as f:
    temp_profile = np.array(f.get("T0")).squeeze(); #
    rho_profile = np.array(f.get("ρ0")).squeeze(); #


## Interpolation

In [62]:
levels = np.linspace(100, 1000, 37)
z_std = np.array(pressure_to_height_std((levels).astype(int) * units.hPa)) *1000.0

# interpolate moisture profile
mean_moist_itp = np.interp(
            np.linspace(100, 1000, 37), np.array([250, 500, 700, 850, 925, 1000]),
            q_reg)

# Interpolate vertical mode
G1_itp = np.interp(z_std, z, G1)# *0.0065;
G2_itp = np.interp(z_std, z, G2)# *0.0065;  

# Interpolate density
rho_profile_itp = np.interp(z_std, z, rho_profile)[:, None]

## Apply LRF to generate radiative heating

### Convert basis with EOFs

In [63]:
# convert G1 and G2 to omega field
G1_omega = -G1 * rho_profile * 9.81
G2_omega = -G2 * rho_profile * 9.81

G1_omega_itp = np.interp(z_std, z, G1_omega)
G2_omega_itp = np.interp(z_std, z, G2_omega)

# decompose vertical mode into PCs
G1_pcs = np.linalg.inv(eof.T @ eof) @ eof.T @ G1_omega_itp
G2_pcs = np.linalg.inv(eof.T @ eof) @ eof.T @ G2_omega_itp

### Compute radiation with LRF

In [64]:
# compute radiation for moisture
lw_moisture = q_lw_lrf @ mean_moist_itp[:,None]
sw_moisture = q_sw_lrf @ mean_moist_itp[:,None]

# compute radiation for temperature
lw_temp1 = t_lw_lrf @ (G1_itp[:,None]*0.0065)
sw_temp1 = t_sw_lrf @ (G1_itp[:,None]*0.0065)
lw_temp2 = t_lw_lrf @ (G2_itp[:,None]*0.0065)
sw_temp2 = t_sw_lrf @ (G2_itp[:,None]*0.0065)

# compute cloud radiation with vertical motion
lw_cld1 = (eof @ (w_lw_lrf @ G1_pcs))[:,None]
sw_cld1 = (eof @ (w_sw_lrf @ G1_pcs))[:,None]
lw_cld2 = (eof @ (w_lw_lrf @ G2_pcs))[:,None]
sw_cld2 = (eof @ (w_sw_lrf @ G2_pcs))[:,None]

## Decompose heating to obtain Coefficients

In [65]:
# stack different modes
modes = np.stack([G1_itp*0.0065, G2_itp*0.0065], axis=-1)

lw_moisture1, lw_moistur2 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_moisture))
sw_moisture1, sw_moistur2 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_moisture))

lw_temp11, lw_temp12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_temp1))
sw_temp11, sw_temp12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_temp1))
lw_temp21, lw_temp22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_temp2))
sw_temp21, sw_temp22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_temp2))

lw_cld11, lw_cld12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_cld1))
sw_cld11, sw_cld12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_cld1))
lw_cld21, lw_cld22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_cld2))
sw_cld21, sw_cld22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_cld2))

## Verification

In [66]:
def plot(
        exact_lw,
        exact_sw,
        approx_lw,
        approx_sw,
        levels, mode
):
    max_limit = np.max(
        [
            np.max(np.abs(exact_lw[1:])),
            np.max(np.abs(approx_lw[1:])),
            np.max(np.abs(exact_sw[1:])),
            np.max(np.abs(approx_sw[1:])),
        ]
    )

    fig = plt.figure(figsize=(6,8))
    plt.plot(exact_lw, levels, color="k", linestyle="-", label="Exact LW")
    plt.plot(approx_lw, levels, color="k", linestyle="--", label="Approx LW")
    plt.plot(exact_sw, levels, color="#1f77b4", linestyle="-", label="Exact SW")
    plt.plot(approx_sw, levels, color="#1f77b4", linestyle="--", label="Approx SW")
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.minorticks_on()
    plt.xlabel("Radiative Heating Rate (K/day)")
    plt.ylabel("Pressure (hPa)")
    plt.xlim(-max_limit*1.1, max_limit*1.1)
    plt.ylim(1000, 100)
    plt.legend(frameon=False)
    plt.savefig(f"/home/b11209013/2025_Research/MSI/Fig/Rad/{mode}.png", dpi=300)
    plt.close()

plot(
    lw_moisture, sw_moisture,
    (lw_moisture1*G1_itp*0.0065 + lw_moistur2*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_moisture1*G1_itp*0.0065 + sw_moistur2*G2_itp*0.0065)[:,None] / rho_profile_itp,
    levels, "moisture"
    )

plot(
    lw_temp1, sw_temp1,
    (lw_temp11*G1_itp*0.0065 + lw_temp12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_temp11*G1_itp*0.0065 + sw_temp12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    levels, "T1"
    )

plot(
    lw_temp2, sw_temp2,
    (lw_temp21*G1_itp*0.0065 + lw_temp22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_temp21*G1_itp*0.0065 + sw_temp22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    levels, "T2"
    )

plot(
    lw_cld1, sw_cld1,
    (lw_cld11*G1_itp*0.0065 + lw_cld12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_cld11*G1_itp*0.0065 + sw_cld12*G2_itp*0.0065)[:,None] / rho_profile_itp,
    levels, "cloud1"
    )

plot(
    lw_cld2, sw_cld2,
    (lw_cld21*G1_itp*0.0065 + lw_cld22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    (sw_cld21*G1_itp*0.0065 + sw_cld22*G2_itp*0.0065)[:,None] / rho_profile_itp,
    levels, "cloud2"
    )